In [ ]:
# Importing libraries
import numpy as np
from scipy.stats import norm

Computing European Call and Put Options Prices using a binomial tree

In [ ]:
# Defining European Call Option in Bionomial Tree funtion
def binomial_tree_call(S_ini, K, T, r, u, d, N):
    dt = T / N # Defining time step
    p = (np.exp(r * dt) - d) / (u - d) # Risk-neutral probabilities
    C = np.zeros([N + 1, N + 1]) #  Call prices
    S = np.zeros([N + 1, N + 1]) # Underlying price

    for i in range(0, N + 1):
        C[N, i] = max(S_ini * (u ** (i)) * (d ** (N - i)) -K, 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))

    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            C[j, i] = np.exp(-r * dt) * (p * C[j + 1, i + 1] + (1 - p) * C[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return C[0, 0], C, S

In [ ]:
# Defining European Put Option in Bionomial Tree funtion
def binomial_tree_put(S_ini, K, T, r, u, d, N):
    dt = T / N # Defining time step
    p = (np.exp(r * dt) - d) / (u - d) # Risk-neutral probabilities
    P = np.zeros([N + 1, N + 1]) #  Put prices
    S = np.zeros([N + 1, N + 1]) # Underlying price

    for i in range(0, N + 1):
        P[N, i] = max(K - (S_ini * (u ** (i)) * (d ** (N - i))), 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))

    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            P[j, i] = np.exp(-r * dt) * (p * P[j + 1, i + 1] + (1 - p) * P[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return P[0, 0], P, S

In [ ]:
# Parameters
S_ini = 100
r = 0.05
sigma = 0.2
T = 3/12 # Months in year
K = 100 # At the money (ATM) strike price
N = 50 # Number of time steps

dt = T / N # Length of one time step
u = np.exp(sigma * np.sqrt(dt)) # Up factor
d = np.exp(-sigma * np.sqrt(dt)) # Down factor

european_call_price, C, S = binomial_tree_call(S_ini, K, T, r, u, d, N)
european_put_price, P, S = binomial_tree_put(S_ini, K, T, r, u, d, N)

print("European Call Option Price: {:.2f}".format(european_call_price))
print("European Put Option Price: {:.2f}".format(european_put_price))

European Call Option Price: 4.60
European Put Option Price: 3.35


Computing the Greek Delta for the European call and European put at time 0

In [ ]:
# Defining the Greek Delta for the European Call function
def call_option_delta(S_ini, K, T, r, u, d, N):
    dt = T / N  # Define time step
    p = (np.exp(r * dt) - d) / (u - d) # Risk neutral probabilities
    C = np.zeros([N + 1, N + 1])  # Call prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    Delta = np.zeros([N, N])  # Delta

    for i in range(0, N + 1):
        C[N, i] = max(S_ini * (u ** (i)) * (d ** (N - i)) - K, 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))

    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            C[j, i] = np.exp(-r * dt) * (p * C[j + 1, i + 1] + (1 - p) * C[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
            Delta [j, i] = (C[j + 1, i + 1] - C[j + 1, i]) / (
                S[j + 1, i + 1] - S[j + 1, i])

    return C[0, 0], Delta[0, 0]

In [ ]:
# Defining the Greek Delta for the Eupropean Put function
def put_option_delta(S_ini, K, T, r, u, d, N):
    dt = T / N  # Define time step
    p = (np.exp(r * dt) - d) / (u - d) # Risk neutral probabilities
    P = np.zeros([N + 1, N + 1])  # Put prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    Delta = np.zeros([N, N])  # Delta

    for i in range(0, N + 1):
        P[N, i] = max(K - (S_ini * (u ** (i)) * (d ** (N - i))), 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))

    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            P[j, i] = np.exp(-r * dt) * (p * P[j + 1, i + 1] + (1 - p) * P[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
            Delta [j, i] = (P[j + 1, i + 1] - P[j + 1, i]) / (
                S[j + 1, i + 1] - S[j + 1, i])

    return P[0, 0], Delta[0, 0]

In [ ]:
# Calculating the Greek Delta for European Call and Put Options
european_call_price, delta_call = call_option_delta(S_ini, K, T, r, u, d, N)
european_put_price, delta_put = put_option_delta(S_ini, K, T, r, u, d, N)

print("The Greek Delta for the European Call Option: {:.2f}".format(delta_call))
print("The Greek Delta for the European Put Option: {:.2f}".format(delta_put))

The Greek Delta for the European Call Option: 0.57
The Greek Delta for the European Put Option: -0.43


Computing the sensitivity of previous put and call option prices to a 5% increase in volatility (from 20% to 25%).

In [ ]:
# Defining the European Call Option Price
def binomial_tree_call(S_ini, K, T, r, u, d, N):
    dt = T / N  # Time step
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probabilities
    C = np.zeros([N + 1, N + 1])  # Call prices

    for i in range(N + 1):
        C[N, i] = max(S_ini * (u ** i) * (d ** (N - i)) - K, 0)

    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            C[j, i] = np.exp(-r * dt) * (p * C[j + 1, i + 1] + (1 - p) * C[j + 1, i])

    return C[0, 0]

# Defining the European Put Option Price
def binomial_tree_put(S_ini, K, T, r, u, d, N):
    dt = T / N  # Time step
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probabilities
    P = np.zeros([N + 1, N + 1])  # Put prices

    for i in range(N + 1):
        P[N, i] = max(K - (S_ini * (u ** i) * (d ** (N - i))), 0)

    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            P[j, i] = np.exp(-r * dt) * (p * P[j + 1, i + 1] + (1 - p) * P[j + 1, i])

    return P[0, 0]

# Defining function to calculate the Vega for Call and Put Options
def vega_sensitivity(S_ini, K, T, r, sigma1, sigma2, N):
    dt = T / N # Define time Step
    u1 = np.exp(sigma1 * np.sqrt(dt)) # Up factor for initial volatility
    d1 = 1 / u1 # Down factor for initial volatility
    u2 = np.exp((sigma1 + sigma2) * np.sqrt(dt)) # Up factor for increased volatility
    d2 = 1 / u2 # Down factor for increased volatility

# Calculating the European Call and Put Options Prices
    call_price_sigma1 = binomial_tree_call(S_ini, K, T, r, u1, d1, N)
    put_price_sigma1 = binomial_tree_put(S_ini, K, T, r, u1, d1, N)

    call_price_sigma2 = binomial_tree_call(S_ini, K, T, r, u2, d2, N)
    put_price_sigma2 = binomial_tree_put(S_ini, K, T, r, u2, d2, N)

# Calculating Sensitivities
    call_vega_sensitivity = (call_price_sigma2 - call_price_sigma1) / sigma2
    put_vega_sensitivity = (put_price_sigma2 - put_price_sigma1) / sigma2

    return call_vega_sensitivity, put_vega_sensitivity

# Parameter
S_ini = 100
K = 100
r = 0.05
T = 3 / 12 # Month in year
N = 50 # Number of time steps

# A 5% increase in volatility (from 20% to 25%)
sigma1 = 0.20
sigma2 = 0.05 # Increment in volatility

# Computing Vega Sensitivities
call_vega_sensitivity, put_vega_sensitivity = vega_sensitivity(S_ini, K, T, r, sigma1, sigma2, N)

print("Vega Sensitivity of the Call Option Price: {:.2f}".format(call_vega_sensitivity))
print("Vega Sensitivity of the Put Option Price: {:.2f}".format(put_vega_sensitivity))

Vega Sensitivity of the Call Option Price: 19.57
Vega Sensitivity of the Put Option Price: 19.57


Computing the American Call and Put Options using a trinomial tree

In [ ]:
# Defining American Call Option in Trionomial Tree funtion
def trinomial_tree_call(S_ini, K, T, r, sigma, N):
    dt = T / N  # Define time step
    u = np.exp(sigma * np.sqrt(2 * dt))  # Up factor
    d = np.exp(-sigma * np.sqrt(2 * dt))  # Down factor
    m = 1  # Middle factor

# Risk-neutral probabilities for a trinomial tree
    p_u = ((np.exp(r * dt) - d) / (u - d)) / 2
    p_d = ((u - np.exp(r * dt)) / (u - d)) / 2
    p_m = 1 - p_u - p_d

# Initializing asset price and option value trees
    price_tree = np.zeros((2 * N + 1, N + 1))
    option_tree = np.zeros((2 * N + 1, N + 1))

# Setting the final nodes for the asset price tree
    for i in range(N + 1):
        for j in range(-i, i + 1, 2):
            price_tree[N + j, i] = S_ini * (u ** (i - abs(j))) * (d ** abs(j))

# Setting the final nodes for the American Call Option Price tree
        option_tree[:, N] = np.maximum(price_tree[:, N] - K, 0)

# Backward induction to compute option values
    for i in range(N - 1, -1, -1):
        for j in range(-i, i + 1, 2):

# Calculating the Call Option Prices using risk-neutral probabilities
            option_tree[N + j, i] = np.exp(-r * dt) * (
                p_u * option_tree[N + j + 1, i + 1] +
                p_d * option_tree[N + j - 1, i + 1] +
                p_m * option_tree[N + j, i + 1]
            )
# Checking American Option feature for early exercise
            option_tree[N + j, i] = max(option_tree[N + j, i], price_tree[N + j, i] - K)

    return option_tree[N, 0], option_tree, price_tree

# Parameters
S_ini = 100
K = [90, 95, 100, 105, 110]  # Strike prices by selecting moneyness of 90%, 95%, ATM, 105%,110%
r = 0.05
sigma = 0.2
T = 3 / 12  # 3 months in years
N = 50  # Number of time steps

# Compute American Call Option Prices for each strike price
call_prices = []
for k in K:
    call_price, call_tree, call_prices_tree = trinomial_tree_call(S_ini, k, T, r, sigma, N)
    call_prices.append(call_price)
    print("American Call Option Price for Strike Price {} is: {:.2f}".format(k, call_price))

American Call Option Price for Strike Price 90 is: 10.00
American Call Option Price for Strike Price 95 is: 5.00
American Call Option Price for Strike Price 100 is: 0.61
American Call Option Price for Strike Price 105 is: 0.10
American Call Option Price for Strike Price 110 is: 0.02


In [ ]:
# Defining American Put Option in Trionomial Tree funtion
def trinomial_tree_put(S_ini, K, T, r, sigma, N):
    dt = T / N  # Define time step
    u = np.exp(sigma * np.sqrt(2 * dt))  # Up factor
    d = np.exp(-sigma * np.sqrt(2 * dt))  # Down factor
    m = 1  # Middle factor

# Risk-neutral probabilities for a trinomial tree
    p_u = ((np.exp(r * dt) - d) / (u - d)) / 2
    p_d = ((u - np.exp(r * dt)) / (u - d)) / 2
    p_m = 1 - p_u - p_d

# Initializing asset price and option value trees
    price_tree = np.zeros((2 * N + 1, N + 1))
    option_tree = np.zeros((2 * N + 1, N + 1))

# Setting the final nodes for the asset price tree
    for i in range(N + 1):
        for j in range(-i, i + 1, 2):
            price_tree[N + j, i] = S_ini * (u ** (i - abs(j))) * (d ** abs(j))

# Setting the final nodes for the American Put Option Price tree
        option_tree[:, N] = np.maximum(K - price_tree[:, N], 0)

# Backward induction to compute option values
    for i in range(N - 1, -1, -1):
        for j in range(-i, i + 1, 2):

# Calculating the Put Option Prices using risk-neutral probabilities
            option_tree[N + j, i] = np.exp(-r * dt) * (
                p_u * option_tree[N + j + 1, i + 1] +
                p_d * option_tree[N + j - 1, i + 1] +
                p_m * option_tree[N + j, i + 1]
            )
# Checking American Option feature for early exercise
            option_tree[N + j, i] = max(option_tree[N + j, i], K - price_tree[N + j, i])

    return option_tree[N, 0], option_tree, price_tree

# Parameters
S_ini = 100
K = [90, 95, 100, 105, 110]  # Strike prices by selecting moneyness of 90%, 95%, ATM, 105%,110%
r = 0.05
sigma = 0.2
T = 3 / 12  # 3 months in years
N = 50  # Number of time steps

# Compute American Put Option Prices for each strike price
put_prices = []
for k in K:
    put_price, put_tree, put_prices_tree = trinomial_tree_put(S_ini, k, T, r, sigma, N)
    put_prices.append(put_price)
    print("American Put Option Price for Strike Price {} is: {:.2f}".format(k, put_price))

American Put Option Price for Strike Price 90 is: 0.00
American Put Option Price for Strike Price 95 is: 0.03
American Put Option Price for Strike Price 100 is: 0.99
American Put Option Price for Strike Price 105 is: 5.00
American Put Option Price for Strike Price 110 is: 10.00
